In [21]:
import nltk
import numpy as np
import random
import pandas as pd
import string # to process standard python strings

In [32]:
f=open("chatbot_faq.csv",'r',errors = 'ignore')

raw=f.read()
raw=raw.lower()# converts to lowercase
print("...", type(raw))
nltk.download('punkt') # first-time use only (puctuation)
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words


... <class 'str'>


[nltk_data] Downloading package punkt to /home/admin1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/admin1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
type(sent_tokens)

list

In [23]:
data = pd.read_csv("chatbot_faq.csv")
data.head()

,Question,DomainIntent,Intent,Answer,Class
0,How long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,How much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,What is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,Can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,How do I get in?,fellowship program,enter,You will require to register for one of our r...,general


In [5]:
# import library for word and sentence tokenize
# from nltk.tokenize import sent_tokenize, word_tokenize
sent_tokens[:2]

['question,domainintent,intent,answer,class\nhow long is the fellowship program?,fellowship program,duration, the program is 4 months on a full-time basis.,general\nhow much does fellowship program cost?,fellowship program,admission_fees, the program is free to the fellows.',
 'you do not have to pay anything for the program.']

In [6]:
word_tokens[:2]

['question', ',']

##### Pre Processing the raw text

In [8]:
# LemTokens will take as input the tokens and return normalized tokens.
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [9]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

##### Generating Response
To generate a response from our bot for input questions, the concept of document similarity will be used. So we begin by importing necessary modules.

From scikit learn library, import the TFidf vectorizer to convert a collection of raw documents to a matrix of TF-IDF features.


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Also, import cosine similarity module from scikit learn library
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    print(".////.",type(sent_tokens))
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    print(".........",type(sent_tokens))
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [31]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: I am glad! You are talking to me
hello
ROBO: hi
good
ROBO: .////. <class 'list'>
......... <class 'list'>
I am sorry! I don't understand you
bye
ROBO: Bye! take care..
